In [8]:
# !pip install mediapipe --user

In [1]:
import numpy as np
import cv2 as cv
import mediapipe as mp
import tensorflow as tf
import os
import time

tf.random.set_seed(42)

In [2]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.8
)
 
# Initializing the drawng utils for drawing the landmarks on image
mp_drawing = mp.solutions.drawing_utils

In [5]:
def get_landmarks(results):
    POSE_LANDMARK_COUNT, HAND_LANDMARK_COUNT, FACE_LANDMARK_COUNT = 33, 21, 468
    pose_landmarks = [0 for _ in range(POSE_LANDMARK_COUNT * 4)]
    left_hand_landmarks = [0 for _ in range(HAND_LANDMARK_COUNT * 3)]
    right_hand_landmarks = [0 for _ in range(HAND_LANDMARK_COUNT * 3)]
    # face_landmarks = [0 for _ in range(FACE_LANDMARK_COUNT * 3)]

    if results.pose_landmarks:
        pose_landmarks = []
        for landmark in results.pose_landmarks.landmark:
            pose_landmarks += [landmark.x, landmark.y, landmark.z, landmark.visibility]

    if results.left_hand_landmarks:
        left_hand_landmarks = []
        for landmark in results.left_hand_landmarks.landmark:
            left_hand_landmarks += [landmark.x, landmark.y, landmark.z]

    if results.right_hand_landmarks:
        right_hand_landmarks = []
        for landmark in results.right_hand_landmarks.landmark:
            right_hand_landmarks += [landmark.x, landmark.y, landmark.z]

    # if results.face_landmarks:
    #     face_landmarks = []
    #     for landmark in results.face_landmarks.landmark:
    #         face_landmarks += [landmark.x, landmark.y, landmark.z]

    return pose_landmarks+left_hand_landmarks+right_hand_landmarks

In [7]:
# def draw_landmarks(image, results):
#     mp_drawing.draw_landmarks(
#       image,
#       results.pose_landmarks,
#       mp_holistic.POSE_CONNECTIONS,
#       mp_drawing.DrawingSpec(
#         color=(255,0,255),
#         thickness=2,
#         circle_radius=2
#       ),
#       mp_drawing.DrawingSpec(
#         color=(255,255,255),
#         thickness=2,
#         circle_radius=2
#       )
#     )
 
#     # Drawing Right hand Land Marks
#     mp_drawing.draw_landmarks(
#       image,
#       results.right_hand_landmarks,
#       mp_holistic.HAND_CONNECTIONS
#     )
 
#     # Drawing Left hand Land Marks
#     mp_drawing.draw_landmarks(
#       image,
#       results.left_hand_landmarks,
#       mp_holistic.HAND_CONNECTIONS
#     )
#     return image

MANUAL-TRAINING

In [15]:
# from IPython.display import clear_output
# TRAINING_DATA_PATH = r'..\dataset\WLASL\start_kit\videos\train'
# FRAME_CNT = 50
# word_map = {}
# X = []
# y = []

# for gloss in os.listdir(TRAINING_DATA_PATH)[:3]:
#     word_map[len(word_map)+1] = gloss
#     word_path = os.path.join(TRAINING_DATA_PATH, gloss)

#     clear_output(wait=True)
#     print('Extracting data of \'{}\'...'.format(gloss))

#     for variation  in os.listdir(word_path):
#         variation_path = os.path.join(word_path, variation)
        
#         cap = cv.VideoCapture(variation_path)
#         total_frames = cap.get(cv.CAP_PROP_FRAME_COUNT)
        
#         if total_frames<FRAME_CNT:
#             cap.release()
#             continue

#         frames_step = total_frames // FRAME_CNT
        
#         i = 10
#         variation_landmarks = []
#         buggy = False
#         while i<FRAME_CNT and i<total_frames:
#             #Set the frame number to (i*frames_step)
#             cap.set(1,  i*frames_step)
#             success, image = cap.read() 
#             if not success:
#                 buggy = True
#                 break
#             image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

#             # retrieve landmarks using holistic model
#             image.flags.writeable = False
#             results = holistic_model.process(image)
#             image.flags.writeable = True

#             draw_landmarks(image, results)

#             cv.imwrite('test_'+gloss+str(i)+'.jpg', image)
        
#             variation_landmarks.append(get_landmarks(results))
#             i+=1
        
#         cap.release()
#         if not buggy:
#             X.append(variation_landmarks)
#             y.append(gloss)
# # cv.destroyAllWindows()

Extracting data of 'africa'...


In [15]:
from IPython.display import clear_output
TESTING_DATA_PATH = r'..\dataset\WLASL\start_kit\videos\test'
FRAME_CNT = 50
word_map = {}
X = []
y = []

for gloss in os.listdir(TESTING_DATA_PATH):
    word_map[len(word_map)+1] = gloss
    word_path = os.path.join(TESTING_DATA_PATH, gloss)

    clear_output(wait=True)
    print(f'Extracting data of \'{gloss}\'...')

    for variation  in os.listdir(word_path):
        variation_path = os.path.join(word_path, variation)
        
        cap = cv.VideoCapture(variation_path)
        total_frames = cap.get(cv.CAP_PROP_FRAME_COUNT)
        
        if total_frames<FRAME_CNT:
            cap.release()
            continue

        frames_step = total_frames // FRAME_CNT
        
        i = 10
        variation_landmarks = []
        buggy = False
        while i<FRAME_CNT and i<total_frames:
            
            #Set the frame number to (i*frames_step)
            cap.set(1,  i*frames_step)
            success, image = cap.read() 
            if not success:
                buggy = True
                break
            
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

            # retrieve landmarks using holistic model
            image.flags.writeable = False
            results = holistic_model.process(image)
            image.flags.writeable = True
        
            variation_landmarks.append(get_landmarks(results))
            i+=1
        
        cap.release()
        if not buggy:
            X.append(variation_landmarks)
            y.append(gloss)

Extracting data of 'your'...


In [16]:
from sklearn.preprocessing import OneHotEncoder

X=np.array(X)       

y = np.array(y)
words_encoded = y.reshape(y.size, 1)
y = OneHotEncoder(sparse=False).fit_transform(words_encoded)

In [17]:
print(X.shape)
y.shape

(296, 40, 258)


(296, 233)

In [21]:
np.save('X_233_Test_batch.npy', X)
np.save('y_233_Test_batch.npy', y)